I do not know this Fort Night of which you speak. Is it the one with the dancing?

In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/fortnite-statistics/Fortnite Statistics.csv')
df['Accuracy (pct)'] = df['Accuracy'].apply(func=lambda x: int(x.replace('%', '')))
df.head()

,Date,Time of Day,Placed,Mental State,Eliminations,Assists,Revives,Accuracy,Hits,Head Shots,Distance Traveled,Materials Gathered,Materials Used,Damage Taken,Damage to Players,Damage to Structures,Accuracy (pct)
0,4/10,6:00 PM,27,sober,2,0,0,23%,14,2,271.08,20,20,272,331,621,23
1,4/10,6:00 PM,45,sober,1,2,0,30%,19,1,396.73,123,30,247,444,998,30
2,4/10,6:00 PM,38,high,3,0,0,30%,32,1,607.80,71,60,176,322,1109,30
3,4/10,7:00 PM,30,high,1,3,0,18%,19,1,714.16,244,10,238,330,4726,18
4,4/10,7:00 PM,16,high,3,1,1,58%,42,18,1140.00,584,150,365,668,2070,58


In [2]:
df.shape

(87, 17)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  87 non-null     object 
 1   Time of Day           87 non-null     object 
 2   Placed                87 non-null     int64  
 3   Mental State          87 non-null     object 
 4   Eliminations          87 non-null     int64  
 5   Assists               87 non-null     int64  
 6   Revives               87 non-null     int64  
 7   Accuracy              87 non-null     object 
 8   Hits                  87 non-null     int64  
 9   Head Shots            87 non-null     int64  
 10  Distance Traveled     87 non-null     float64
 11  Materials Gathered    87 non-null     int64  
 12  Materials Used        87 non-null     int64  
 13  Damage Taken          87 non-null     int64  
 14  Damage to Players     87 non-null     int64  
 15  Damage to Structures  87 

In [4]:
df.nunique()

Date                     7
Time of Day             13
Placed                  41
Mental State             2
Eliminations             9
Assists                  7
Revives                  5
Accuracy                38
Hits                    52
Head Shots              19
Distance Traveled       85
Materials Gathered      77
Materials Used          30
Damage Taken            74
Damage to Players       84
Damage to Structures    85
Accuracy (pct)          38
dtype: int64

In [5]:
from plotly.express import scatter
scatter(data_frame=df, x='Distance Traveled', y='Damage to Players', color='Mental State', size='Accuracy (pct)',).show()